Nous allons détailler les fonctions dans ce notebook.

### IMPORTS

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from pathlib import Path
import os
# from webdriver_manager.chrome import ChromeDriverManager
# import chromedriver_binary
import re
import csv

In [2]:
DRIVER_PATH_chromedriver = str(Path('../chromedriver').resolve())
browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)

/tmp/ipykernel_562/2348712840.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)


In [3]:
regex_expression = "</span> #.* in Kindle Store " # utile pour regex

### URLs exemple

In [4]:
url_kindle_accueil = "https://www.amazon.com/b?node=154606011"
url_kindle_cat_art_and_photo ="https://www.amazon.com/s?bbn=154606011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011&dc&qid=1648440703&rnid=154606011&ref=lp_154606011_nr_n_0"
url_kindle_cat_art_and_photo_subcat_music_p1 = "https://www.amazon.com/s?rh=n%3A156302011&fs=true&ref=lp_156302011_sar"

url_kindle_cat_art_and_photo_subcat_umusic_p2 = "https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=2&qid=1648440846&ref=sr_pg_2"
url_kindle_cat_art_and_photo_subcat_umusic_p3  ="https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=3&qid=1648440968&ref=sr_pg_3"

In [5]:
url_livre1="https://www.amazon.com/Last-Goddess-Novel-Kate%C5%99ina-Tu%C4%8Dkov%C3%A1-ebook/dp/B09BZ8L74Q/ref=pd_sbs_sccl_1_3/131-3814768-2928133?pd_rd_w=eOyFv&pf_rd_p=3676f086-9496-4fd7-8490-77cf7f43f846&pf_rd_r=XXEVJ89KR0WC7CQF9791&pd_rd_r=ff1f8e44-e48e-4dcc-8716-a64d5675ebe6&pd_rd_wg=tAIJl&pd_rd_i=B09BZ8L74Q&psc=1"
url_livre2="https://www.amazon.com/gp/product/B09K7J47Y8/ref=s9_acss_bw_cg_bsmafr_1b1_w?pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-13&pf_rd_r=EX2QJTNDW0DCKTRX8KFT&pf_rd_t=101&pf_rd_p=5074797f-6c32-4283-a5a7-526fd1198645&pf_rd_i=14413390011"
url_livre3="https://www.amazon.com/Healthy-Smoothie-Cookbook-Breakfast-Kid-Friendly-ebook/dp/B083QK3DT9/ref=sr_1_12?keywords=smoothie+recipe+book&qid=1651700938&s=digital-text&sprefix=smoothie%2Cdigital-text%2C236&sr=1-12"
url_yumkasi = "https://www.amazon.com/Minoru-Yamasaki-Humanist-Architecture-Modernist-ebook/dp/B0798MHV9R/ref=sr_1_17?qid=1652127157&rnid=157630011&s=digital-text&sr=1-17"

# Ouverture du navigateur

avant bien lancer le programme X launch sur windows en cochant sur disable

In [6]:
DRIVER_PATH_chromedriver = str(Path('../chromedriver').resolve())
browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)
browser.get(url_kindle_accueil)

/tmp/ipykernel_562/2347370238.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)


avec le programme ci-après j arrive à lancer chrome depuis notebook !! yes

>### il faut réussir à faire différents programme:
>   - sur la page d'un livre ( en input un url) trouver le livre , l'auteur, le ranking
>   - sur une page, trouver tout les livres de la pages
>   - sur une catégorie trouver les pages de catégories

# Fonction 1 : avec url d'un livre , retourner ranking des ventes, le nom l'auteur 

In [7]:
def encrypter(url,titre,rang,prix):
    dictionnary = [{'url':url ,'titre':titre,  'rang':rang ,'prix':prix }]
    print(dictionnary)
    with open('results.csv','a') as f :                        # ouvre le fichier result.csv
        fields=['titre','rang','prix','url']                  # permet de faire un    
        writer = csv.DictWriter(f,fieldnames = fields)
        writer.writerows(dictionnary)


In [13]:
def encryper_depuis_url_page_livre(url):
    browser.get(url)
    code_de_la_page = browser.page_source
    soup = BeautifulSoup(code_de_la_page , 'lxml')
    rang_livre = soup.find_all( {'span'  : 'a-list-item'})
    # URL du livre OK
    url = url 
    # titre du livre
    titre_avec_virgule = soup.find('span'  , {'class': 'a-size-extra-large'}).string
    titre = titre_avec_virgule.replace(",", " ")
    #rang du livre OK
    rang = str(re.compile(regex_expression).search(str(rang_livre))).split()[5].replace("#","").replace(",","") 
    #prix du livre
    if soup.find('span' , class_ = 'a-price-whole') is not None:
        prix = soup.find('span' , class_ = 'a-price-whole').text.strip(' ').strip('.')
    else:
        prix = None
    encrypter(url,titre,rang,prix)
    

In [14]:
#TEST
encryper_depuis_url_page_livre(url_livre3)

[{'url': 'https://www.amazon.com/Healthy-Smoothie-Cookbook-Breakfast-Kid-Friendly-ebook/dp/B083QK3DT9/ref=sr_1_12?keywords=smoothie+recipe+book&qid=1651700938&s=digital-text&sprefix=smoothie%2Cdigital-text%2C236&sr=1-12', 'titre': ' The Healthy Smoothie Cookbook: Breakfast Smoothie  Body Cleansing Smoothies  Digestive Smoothies  Kid-Friendly Smoothies  Low-Fat Smoothies  Best Protein Smoothies  Easy to Make Weight loss Smoothies ', 'rang': '194816', 'prix': '6'}]


# Sur une page récupérer les données : de classement de vente , nom du livre catégorie

In [15]:
books_html = soup.find_all("article", class_="product_pod")
len(books_html)

0

In [16]:
url = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&qid=1651987607&rnid=157630011&ref=sr_nr_n_1"
url_page_1 = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=1&qid=1651987613&rnid=157630011&ref=sr_pg_2"
url_page_2 = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=2&qid=1651987613&rnid=157630011&ref=sr_pg_2"
url_page_3     = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=3&qid=1651987613&rnid=157630011&ref=sr_pg_3"
url_page_3bis  = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=3&qid=1651987793&rnid=157630011&ref=sr_pg_3"


In [17]:
listes_des_livres=[]
browser.get(url_page_2) #navigue sur une page avec tous les lien du site
html_page_listing = browser.page_source # permet de prendre le code html de la page 
soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} )
#tous les livres de la pages
for livre in livres:
     url_du_livre = 'https://www.amazon.com'+livre.h2.a.get('href')
     encryper_depuis_url_page_livre(url_du_livre)
    # le numéro ASIN correspeond à la ref du livre sur amazon
# ASIN=B098652LH8P

[{'url': 'https://www.amazon.com/Four-Books-Architecture-Dover-Book-ebook/dp/B00CWR4LIG/ref=sr_1_17?qid=1653250859&rnid=157630011&s=digital-text&sr=1-17', 'titre': ' The Four Books of Architecture (Dover Architecture Book 1) ', 'rang': '757920', 'prix': '8'}]
[{'url': 'https://www.amazon.com/Frank-Lloyd-Wright-Penguin-Lives-ebook/dp/B00188V7R8/ref=sr_1_18?qid=1653250859&rnid=157630011&s=digital-text&sr=1-18', 'titre': ' Frank Lloyd Wright: A Life (Penguin Lives) ', 'rang': '170831', 'prix': '9'}]
[{'url': 'https://www.amazon.com/Man-Glass-House-Johnson-Architect-ebook/dp/B07DTC5N92/ref=sr_1_19?qid=1653250859&rnid=157630011&s=digital-text&sr=1-19', 'titre': ' The Man in the Glass House: Philip Johnson  Architect of the Modern Century ', 'rang': '580361', 'prix': None}]
[{'url': 'https://www.amazon.com/Plagued-Fire-Dreams-Furies-Wright-ebook/dp/B07MLP6MD5/ref=sr_1_20?qid=1653250859&rnid=157630011&s=digital-text&sr=1-20', 'titre': ' Plagued by Fire: The Dreams and Furies of Frank Lloyd Wr

AttributeError: 'NoneType' object has no attribute 'string'

In [28]:
url_pb = "https://www.amazon.com/Quer%C3%ADamos-Calatrava-arquitect%C3%B3nicos-seducci%C3%B3n-Cr%C3%B3nicas-ebook/dp/B01LZXUQNV/ref=sr_1_20?qid=1652824900&rnid=157630011&s=digital-text&sr=1-20"

In [36]:
encryper_depuis_url_page_livre(url_pb)

[{'url': 'https://www.amazon.com/Quer%C3%ADamos-Calatrava-arquitect%C3%B3nicos-seducci%C3%B3n-Cr%C3%B3nicas-ebook/dp/B01LZXUQNV/ref=sr_1_20?qid=1652824900&rnid=157630011&s=digital-text&sr=1-20', 'titre': ' Queríamos un Calatrava. Viajes arquitectónicos por la seducción y el repudio (Crónicas nº 112) (Spanish Edition) ', 'rang': '1004938', 'prix': None}]


In [27]:
url_du_livre = 'https://www.amazon.com'+livres[2].h2.a.get('href')

In [28]:
url_du_livre

'https://www.amazon.com/Arsenal-Exclusion-Inclusion-Interboro-Partners-ebook/dp/B0989XW388/ref=sr_1_19?qid=1652386827&rnid=157630011&s=digital-text&sr=1-19'

In [26]:
browser.get(url_du_livre)

In [29]:
trouver_rang("https://www.amazon.com/Minoru-Yamasaki-Humanist-Architecture-Modernist-ebook/dp/B0798MHV9R/ref=sr_1_17?qid=1652130424&rnid=157630011&s=digital-text&sr=1-17")

'1230423'

# Fonction permettant d'incrémenter une page web
    

In [38]:
#def lire_page_suivante(url):
url_pour_suivre ="https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=2&qid=1652385543&ref=sr_pg_2"


1 - réussir à couper la chaine de caractère à gauche de "&page="

In [42]:
i = 1

In [46]:
url_pour_suivre.split("&page=")[0] + "&page=" + str(i)

'https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=1'

In [18]:
url_a_incrementer = "https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=2&qid=1653250959&ref=sr_pg_2"


In [24]:
url_a_incrementer.split("&page=")[1].split("&qid=")[0]

'2'

In [48]:
def incrementer_url(url):
    url_gauche =  url.split("&page=")[0]+"&page="  # trouve la première partie de la chaine à gauche
    numero_de_la_page = str(int((url.split("&page=")[1].split("&qid=")[0]))+1) #trouve la partie numérique, convertie en nombre et ajoute 1
    return(str(url_gauche + numero_de_la_page) )

In [49]:
incrementer_url("https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=3")

'https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=4'

In [52]:
url_test = "https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=3"
i=0
while i < 10 :
    url_test = incrementer_url(url_test)
    print(url_test)
    i = i+1

https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=4
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=5
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=6
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=7
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=8
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=9
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=10
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=11
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=12
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=13


In [49]:
listes_des_livres=[]
#tous les livres de la pages
for i in range(10):
    browser.get( url_pour_suivre.split("&page=")[0] + "&page=" + str(i)) #navigue sur une page avec tous les liens du site
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} )
    for livre in livres:
        url_du_livre = 'https://www.amazon.com'+livre.h2.a.get('href')
        encryper_depuis_url_page_livre(url_du_livre) 
   

[{'url': 'https://www.amazon.com/Place-My-Own-Architecture-Daydreams-ebook/dp/B00BE2568E/ref=sr_1_1?qid=1652827066&s=digital-text&sr=1-1', 'titre': ' A Place of My Own: The Architecture of Daydreams ', 'rang': '263356', 'prix': '10'}]
[{'url': 'https://www.amazon.com/Million-Stuffed-Shark-Economics-Contemporary-ebook/dp/B0077QZ208/ref=sr_1_2?qid=1652827066&s=digital-text&sr=1-2', 'titre': ' The $12 Million Stuffed Shark: The Curious Economics of Contemporary Art ', 'rang': '261992', 'prix': '1'}]
[{'url': 'https://www.amazon.com/Gardner-Heist-Worlds-Largest-Unsolved-ebook/dp/B001TJ2YF0/ref=sr_1_3?qid=1652827066&s=digital-text&sr=1-3', 'titre': " The Gardner Heist: The True Story of the World's Largest Unsolved Art Theft ", 'rang': '304142', 'prix': '12'}]
[{'url': 'https://www.amazon.com/Stoicism-Stoic-Philosophy-Inner-Happiness-ebook/dp/B086RGHJ2K/ref=sr_1_4?qid=1652827066&s=digital-text&sr=1-4', 'titre': ' Stoicism: How to Use Stoic Philosophy to Find Inner Peace and Happiness ', 'ra

[{'url': 'https://www.amazon.com/Sandfuture-Justin-Beal-ebook/dp/B08WRCYX6Z/ref=sr_1_18?qid=1652827205&s=digital-text&sr=1-18', 'titre': ' Sandfuture ', 'rang': '1065095', 'prix': '9'}]
[{'url': 'https://www.amazon.com/Frank-Lloyd-Wright-Penguin-Lives-ebook/dp/B00188V7R8/ref=sr_1_19?qid=1652827205&s=digital-text&sr=1-19', 'titre': ' Frank Lloyd Wright: A Life (Penguin Lives) ', 'rang': '578513', 'prix': '14'}]
[{'url': 'https://www.amazon.com/Quer%C3%ADamos-Calatrava-arquitect%C3%B3nicos-seducci%C3%B3n-Cr%C3%B3nicas-ebook/dp/B01LZXUQNV/ref=sr_1_20?qid=1652827205&s=digital-text&sr=1-20', 'titre': ' Queríamos un Calatrava. Viajes arquitectónicos por la seducción y el repudio (Crónicas nº 112) (Spanish Edition) ', 'rang': '1004938', 'prix': None}]
[{'url': 'https://www.amazon.com/Cat%C3%A1logo-Arquitectura-Movimiento-Moderno-Spanish-ebook/dp/B09TFQLKQV/ref=sr_1_21?qid=1652827205&s=digital-text&sr=1-21', 'titre': ' Catálogo Arquitectura Movimiento Moderno Perú (Spanish Edition) ', 'rang': 

[{'url': 'https://www.amazon.com/You-Say-Brick-Life-Louis-ebook/dp/B01KTL87RW/ref=sr_1_51?qid=1652827335&s=digital-text&sr=1-51', 'titre': ' You Say to Brick: The Life of Louis Kahn ', 'rang': '387589', 'prix': '17'}]
[{'url': 'https://www.amazon.com/Architecture-Frank-Lloyd-Wright-Fourth-ebook/dp/B07239RCRV/ref=sr_1_52?qid=1652827335&s=digital-text&sr=1-52', 'titre': ' The Architecture of Frank Lloyd Wright, Fourth Edition: A Complete Catalog ', 'rang': '1383271', 'prix': '9'}]
[{'url': 'https://www.amazon.com/Antoni-Gaud%C3%AD-Architect-Temporis-Collection-ebook/dp/B016XN11IM/ref=sr_1_53?qid=1652827335&s=digital-text&sr=1-53', 'titre': ' Antoni Gaudí: Architect and Artist (Temporis Collection) ', 'rang': '1569426', 'prix': '2'}]
[{'url': 'https://www.amazon.com/Create-Prosper-Clients-Freelance-Business-ebook/dp/B086WP4MJM/ref=sr_1_54?qid=1652827335&s=digital-text&sr=1-54', 'titre': ' Create And Prosper: How To Find Your Dream Clients And Build A Freelance Business You Love ', 'rang':

KeyboardInterrupt: 

In [54]:
import pandas as pd

In [55]:
data = pd.read_csv("results.csv")

In [56]:
data

,Titre,rang,prix,url
0,A Place of My Own: The Architecture of Daydre...,263356,10.0,https://www.amazon.com/Place-My-Own-Architectu...
1,The $12 Million Stuffed Shark: The Curious Ec...,261992,1.0,https://www.amazon.com/Million-Stuffed-Shark-E...
2,The Gardner Heist: The True Story of the Worl...,304142,12.0,https://www.amazon.com/Gardner-Heist-Worlds-La...
3,Stoicism: How to Use Stoic Philosophy to Find...,118329,3.0,https://www.amazon.com/Stoicism-Stoic-Philosop...
4,Confessions Of an Art Addict,581361,4.0,https://www.amazon.com/Confessions-Art-Addict-...
...,...,...,...,...
112,Sandfuture,1156955,48.0,https://www.amazon.com/Sandfuture-Justin-Beal-...
113,You Say to Brick: The Life of Louis Kahn,855637,8.0,https://www.amazon.com/You-Say-Brick-Life-Loui...
114,Where Are the Women Architects? (Places Books...,814900,10.0,https://www.amazon.com/Where-Women-Architects-...
115,Wright Sites: A Guide to Frank Lloyd Wright P...,754775,33.0,https://www.amazon.com/Wright-Sites-Guide-Publ...
